Date offset and shift dates

In [2]:
import pandas as pd
from pathlib import Path
import shutil
import stat
import numpy as np
from dataforge import tools
from jdc_utils.transforms import deidentify
from git import Repo

In [3]:
#https://stackoverflow.com/questions/4829043/how-to-remove-read-only-attrib-directory-with-python-in-windows
def on_rm_error( func, path, exc_info):
    # path contains the path of the file that couldn't be removed
    # let's just assume that it's read-only and unlink it.
    os.chmod( path, stat.S_IWRITE )
    os.unlink( path )

In [4]:
df = pd.read_csv("../tests/data/transform-tests/test-1-long-format.csv")
mapping_file = Path("../tests/data/transform-tests/test_mappings.csv").resolve().as_posix()

In [5]:
# %%writefile -a ../tests/transforms/test_deidentify.py

""" test shift dates without git bare repo """
deidentify.shift_dates(
    df=df,
    id_column='record_id',
    date_columns='date_var',
    map_file=mapping_file)

    
os.remove('days_for_shift_date.csv')

In [6]:
# %%writefile -a ../tests/transforms/test_deidentify.py

"""test shift dates with git bare repo (ie remote url)"""


# create a git bare repo
test_remote_url = Path(mapping_file).with_suffix('.git').as_posix()
repo = Repo.init(test_remote_url,bare=True)
repo.close()

#run shift dates with git bare repo url
dates1 = deidentify.shift_dates(
    df=df,
    id_column='record_id',
    date_columns='date_var',
    map_file=mapping_file,
    map_url=test_remote_url)

# show evidence that it has been added to history
repo = Repo(test_remote_url)
assert len(list(repo.iter_commits()))==1
repo.close()
assert (df['date_var'].dt.strftime('%Y%m%d')!=dates1["shifted_date_var"]).all()
# delete git bare repo and tmp/git
shutil.rmtree("tmp/git",onerror=on_rm_error)
shutil.rmtree(repo.working_dir,onerror=on_rm_error)

In [21]:
repo?

Type:           Repo
String form:    <git.repo.base.Repo 'C:\\Users\\kranz-michael\\projects\\jdc-utilities\\tests\\data\\transform-tests\\test_mappings.git'>
File:           c:\users\kranz-michael\miniconda3_v2\envs\jdc-utils\lib\site-packages\git\repo\base.py
Docstring:     
Represents a git repository and allows you to query references,
gather commit information, generate diffs, create and clone repositories query
the log.

The following attributes are worth using:

'working_dir' is the working directory of the git command, which is the working tree
directory if available or the .git directory in case of bare repositories

'working_tree_dir' is the working tree directory, but will raise AssertionError
if we are a bare repository.

'git_dir' is the .git repository directory, which is always set.
Init docstring:
Create a new Repo instance

:param path:
    the path to either the root git directory or the bare git repo::

        repo = Repo("/Users/mtrier/Development/git-python")
    

In [19]:
shutil.rmtree(repo.working_dir)

PermissionError: [WinError 5] Access is denied: 'C:\\Users\\kranz-michael\\projects\\jdc-utilities\\tests\\data\\transform-tests\\test_mappings.git\\objects\\c4\\4ce4998dadfa486f86fa0b37e539e4cb6ab3b4'